In order to run the code in parallel, we first need to add some julia processes.

In [12]:
using Distributed
addprocs(7)
nprocs()

11

# Shared memory parallelization of block banded matrices

The objective is to parallelize block banded matrices using a shared memory
model. Block banded matrices can be parameterized to accept different kinds of
storage back-ends. Among other things, it means block banded matrices can be
parallelized simply by telling them to use a `SharedArray` as storage:

In [ ]:
@everywhere begin
    using Pkg
    Pkg.activate(pwd())
    using BlockBandedMatrices: BandedBlockBandedMatrix, _BandedBlockBandedMatrix
    using BlockBandedMatrices: BandedBlockBandedSizes
    using BlockArrays: PseudoBlockArray, Block, nblocks
    using SharedArrays: SharedArray

    const SharedPseudoBlock = 
        PseudoBlockArray{T, 2, SharedArray{T, 2}, B} where {T, B}
    const SharedBandedBlockBandedMatrix =
        BandedBlockBandedMatrix{T, SharedPseudoBlock{T, B}} where {T, B}
end

So we have a shared array at the very bottom. It holds the actual data. On it,
we layer an abstraction, the `PseudoBlockArray` that allows us to view the
shared array as an array of blocks. On top of that, the
`BandedBlockBandedMatrix` adds another layer of abstraction to the view the
block array as banded matrix of block banded matrices.

We can now create an overloaded function to initialize the array:

In [ ]:
@everywhere function SharedBandedBlockBandedMatrix{T}(
        ::UndefInitializer, bs::BandedBlockBandedSizes; kwargs...) where T
    shared = SharedArray{T}(size(bs); kwargs...)
    block = PseudoBlockArray(shared, bs.data_block_sizes)
    _BandedBlockBandedMatrix(block, bs)
end

@everywhere function SharedBandedBlockBandedMatrix{T}(
        ::UndefInitializer,
        dims::NTuple{2, AbstractVector{Int}},
        lu::NTuple{2, Int},
        λμ::NTuple{2, Int};
        kwargs...
) where T
    blocksizes = BandedBlockBandedSizes(dims..., lu..., λμ...)
    SharedBandedBlockBandedMatrix{T}(undef, blocksizes; kwargs...)
end

To illustrate this, let's create a banded block banded matrix with blocks
composed of 10x10 matrices (`n` and `m`), with the band of blocks from `l` to
`u` and each block banded from `λ` to `μ`.

In [ ]:
N, n = 2, 3
l, u, λ, μ = 1, 0, 0, 2
M, m = N + 1, n + 1
A = SharedBandedBlockBandedMatrix{Float64}(undef, (repeat([n], N), repeat([m], M)), (l, u), (λ, μ))

But what we really want to do is to populate the matrix in parallel. We can do
this by allocating blocks for each process to write to:

In [ ]:
@everywhere begin
    """Populates the shared banded block banded matrix in parallel"""
    function populate!(
            block_populate!::Function, A::SharedBandedBlockBandedMatrix)
        @sync begin
            for (p, proc) in enumerate(procs(A.data.blocks))
                @async remotecall_wait(populate!, proc, block_populate!, A, p)
            end
        end
        A
    end

    """Populates the blocks of A that are assigned to process p."""
    function populate!(
            block_populate!::Function, A::SharedBandedBlockBandedMatrix, p)
        n = nmutableblocks(A)

        m = length(procs(A.data.blocks))
        start = (n ÷ m) * (p - 1) + min((n % m), p - 1)
        stop = (n ÷ m) * p + min((n % m), p)

        k = 0
        for i in 1:nblocks(A, 1), j in max(i - A.l, 1):min(i + A.u, nblocks(A, 2))
            if k >= stop
                break
            elseif k >= start
                block_populate!(view(A, Block(i, j)), i, j)
            end
            k += 1
        end
        A
    end

    """Counts the number of blocks in the mutable bands."""
    function nmutableblocks(A::SharedBandedBlockBandedMatrix)
        n = nblocks(A, 1)
        for i in 1:nblocks(A, 1)
            n += min(i + A.u, nblocks(A, 2)) - max(i - A.l, 1)
        end
        n
    end
end

The code proceeds in a cascade of three steps. First we have a functions to
signal workers to start working. Then we have a funtion that will run on each
worker process. It simply parcels out the list of mutable blocks in the banded
matrix and figures out which are assigned to it's process id. For each assigned
block, it then calls the third function in the cascade, to actually populate the
given block.

Performance-minded users will notice that this simple-minded implementation runs
on idle for a fair bit of the loop over blocks. But in practice, this is a small
waste compared to latency issues we will soon dive into.

The magic happens in the `@sync` for loop. In practice, each `@async` loop item
generates a task and schedules it to be sent and executed by a process.  Then,
right before exiting the `@sync` code-block, the tasks are sent over to the
processes, and the code waits until the tasks are finished running.

In practice, usage looks like this:

In [ ]:
populate!(A) do block, i, j
    # block: a view of the block to populate
    # i, j: indices of the block
    for x in 1:size(block, 1), y in max(x - A.λ, 1):min(x + A.μ, size(block, 2))
        block[x, y] = 1
    end
end

For comparison we also construct the serial version of populate, as well as a
simple scheme to fill blocks with ones:

In [ ]:
@everywhere function populate!(
        A::BandedBlockBandedMatrix, block_populate!::Function)
    for i in 1:nblocks(A, 1), j in max(i - A.l, 1):min(i + A.u, nblocks(A, 2))
        block_populate!(view(A, Block(i, j)), i, j)
    end
    A
end

@everywhere function simplefill!(A::BandedBlockBandedMatrix)
    populate!(A) do bk, i, j
        for x in 1:size(bk, 1), y in max(x - A.λ, 1):min(x + A.μ, size(bk, 2))
            bk[x, y] = 1
        end
    end
end

And now we can run benchmarks:

In [ ]:
using BenchmarkTools

N, n = 8, 3
l, u, λ, μ = 1, 2, 0, 1
M, m = 6, n
A = SharedBandedBlockBandedMatrix{Float64}(undef, (repeat([n], N), repeat([m], M)), (l, u), (λ, μ))
nmutableblocks(A)
@benchmark simplefill!($A)

In [ ]:
B = BandedBlockBandedMatrix{Float64}(undef, (repeat([n], N), repeat([m], M)), (l, u), (λ, μ))
@benchmark simplefill!($B)

And that's not a win for the multi-core team... But we do not always expect a
win in every situation. Surely, the performance depends on the size of the
matrix? So let's try a few:

In [ ]:
n, l, u, λ, μ = 500, 1, 2, 2, 1
serial, parallel, Ns = [], [], [10, 50, 100, 150, 200, 300, 500]
for N = Ns
    A = SharedBandedBlockBandedMatrix{Float64}(
            undef, (repeat([n], N), repeat([n], N)), (l, u), (λ, μ))
    bench = @benchmark simplefill!($A)
    push!(parallel, bench)

    B = BandedBlockBandedMatrix{Float64}(
            undef, (repeat([n], N), repeat([n], N)), (l, u), (λ, μ))
    bench = @benchmark simplefill!($B)
    push!(serial, bench)
end

In [ ]:
using VegaLite, DataFrames, Statistics
DataFrame(
    data = vcat(
        getfield.(mean.(serial), :time) ./ 1000,
        getfield.(mean.(parallel), :time) ./ 1000,
    ),
    n = repeat(Ns, 2),
    method=vcat(
        repeat([:serial], length(Ns)),
        repeat([:parallel], length(Ns))
    )
) |> @vlplot(mark={:line, point=:true}, x=:n, y=:data, color=:method)